<a href="https://colab.research.google.com/github/ferranfont/Introduccion_al_trading_algoritmico/blob/main/Quant_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**QUANTITATIVE MOMENTUM SYSTEM**

In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importamos todos los tickets del SP500
stocks = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sp_500_stocks.csv')
stocks

In [ ]:
# Hacemos un API a un end point de momentum
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol='BA'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

In [ ]:
# Consultamos un campo en particular
data['month1ChangePercent']*100

In [ ]:
round(data['peRatio'],1)

In [ ]:
my_colums = ['Ticker','Descripción', 'rentabilidad_1_año','PER','n_acciones_a_comprar']
final_dataframe = pd.DataFrame(columns=my_colums)
final_dataframe

In [ ]:
final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'], 
                                                  index =my_colums), 
                                        ignore_index = True)
final_dataframe

In [ ]:
for i in acciones['Ticker'][:80]: #para no ralentizar el proceso lo haremos tan sólo para las primeros 80 símbolos [:80]
  api_url = f'https://sandbox.iexapis.com/stable/stock/{i}/quote?token=Tpk_059b97af715d417d9f49f50b51b1c448'
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
                                        pd.Series([i, 
                                                   data[''], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index =my_colums), 
                                        ignore_index = True)
